# Example of TwoTower training/inference

This example uses `ParquetModule` for data loading and processing. 

**It requires setting environment variables OMP_NUM_THREADS and ARROW_IO_THREADS to match the number of available CPUs.**

In [1]:
%env OMP_NUM_THREADS=8
%env ARROW_IO_THREADS=8

env: OMP_NUM_THREADS=8
env: ARROW_IO_THREADS=8


In [2]:
import lightning as L
import pandas as pd

L.seed_everything(42);

Seed set to 42


## Preparing data
In this example, we will be using the MovieLens dataset, namely the 1m subset. It's demonstrated a simple case, so only item ids will be used as model input.

**NOTE**

Current implementation of TwoTower is able to handle item and interactions features. It does not take into account user features. 


In [3]:
interactions = pd.read_csv("./data/ml1m_ratings.dat", sep="\t", names=["user_id", "item_id","rating","timestamp"])
interactions = interactions.drop(columns=["rating"])

item_features = pd.read_csv("./data/ml1m_items.dat", sep="\t", names=["item_id", "title", "genres"])
item_features = item_features.drop(columns=["title", "genres"])

In [4]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,timestamp
1000138,6040,858,0
1000153,6040,2384,1
999873,6040,593,2
1000192,6040,2019,3
1000007,6040,1961,4
...,...,...,...
825793,4958,2399,446
825438,4958,1407,447
825731,4958,2634,448
825724,4958,3264,449


### Encode catagorical data.
To ensure all categorical data is fit for training, it needs to be encoded using the `LabelEncoder` class. Create an instance of the encoder, providing a `LabelEncodingRule` for each categorcial column in the dataset that will be used in model. Note that ids of users and ids of items are always used.

Let's train the encoder for the `item_id` column using the item features, because some items may be missing from interactions. The `user_id` column will be trained using interactions.

In [5]:
from replay.preprocessing import LabelEncoder, LabelEncodingRule

encoder = LabelEncoder(
    [
        LabelEncodingRule("user_id"),
        LabelEncodingRule("item_id"),
    ]
)
encoder.rules[0].fit(interactions)
encoder.rules[1].fit(item_features)
encoded_interactions = encoder.transform(interactions)
encoded_interactions

,timestamp,user_id,item_id
0,0,6039,847
1,1,6039,2315
2,2,6039,589
3,3,6039,1950
4,4,6039,1892
...,...,...,...
1000204,446,4957,2330
1000205,447,4957,1384
1000206,448,4957,2565
1000207,449,4957,3195


In [6]:
item_features_encoded = encoder.rules[1].transform(item_features)
item_features_encoded

,item_id
0,0
1,1
2,2
3,3
4,4
...,...
3878,3878
3879,3879
3880,3880
3881,3881


### Split interactions into the train, validation and test datasets using LastNSplitter
We use widespread splitting strategy Last-One-Out. We filter out cold items and users for simplicity.

In [7]:
from replay.splitters import LastNSplitter

splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
    drop_cold_users=True,
    drop_cold_items=True
)

test_events, test_gt = splitter.split(encoded_interactions)
validation_events, validation_gt = splitter.split(test_events)
train_events = validation_events

### Dataset preprocessing ("baking")
SasRec expects each user in the batch to provide their events in form of a sequence. For this reason, the event splits must be properly processed using the `groupby_sequences` function provided by RePlay.

In [8]:
from replay.data.nn.utils import groupby_sequences


def bake_data(full_data):
    grouped_interactions = groupby_sequences(events=full_data, groupby_col="user_id", sort_col="timestamp")
    return grouped_interactions


train_events = bake_data(train_events)

validation_events = bake_data(validation_events)
validation_gt = bake_data(validation_gt)

test_events = bake_data(test_events)
test_gt = bake_data(test_gt)

train_events

,user_id,timestamp,item_id
0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[3117, 1250, 1009, 1672, 2271, 1768, 3339, 118..."
1,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1180, 1192, 1199, 2648, 1273, 2874, 1207, 315..."
2,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[589, 2789, 1899, 3465, 1407, 1892, 1246, 1358..."
3,3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1192, 1081, 3458, 476, 3399, 257, 1180, 1178,..."
4,4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[2648, 907, 896, 352, 1230, 2119, 2789, 1111, ..."
...,...,...,...
6035,6035,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1672, 1814, 3369, 2307, 2359, 2503, 2423, 278..."
6036,6036,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[693, 1813, 3439, 1959, 1247, 558, 847, 3079, ..."
6037,6037,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[3327, 908, 1192, 2077, 1366, 352, 1063, 1132,..."
6038,6038,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[109, 279, 1998, 1211, 918, 3064, 935, 3019, 2..."


To ensure we don't have unknown users in ground truth, we join validation events and validation ground truth (also join test events and test ground truth correspondingly) by user ids to leave only the common ones.  

In [9]:
def add_gt_to_events(events_df, gt_df):
    gt_to_join = gt_df[["user_id", "item_id"]].rename(columns={"item_id": "ground_truth"})

    events_df = events_df.merge(gt_to_join, on="user_id", how="inner")
    return events_df

validation_events = add_gt_to_events(validation_events, validation_gt)
test_events = add_gt_to_events(test_events, test_gt)

In [10]:
from pathlib import Path

data_dir = Path("temp/data/")
data_dir.mkdir(parents=True, exist_ok=True)

TRAIN_PATH = data_dir / "train.parquet"
VAL_PATH = data_dir / "val.parquet"
TEST_PATH = data_dir / "test.parquet"

PATH_ENCODED_FEATURES = data_dir / "item_features_encoded.parquet"

ENCODER_PATH = data_dir / "encoder"

In [ ]:
train_events.to_parquet(TRAIN_PATH)
validation_events.to_parquet(VAL_PATH)
test_events.to_parquet(TEST_PATH)

item_features_encoded[["item_id"]].to_parquet(PATH_ENCODED_FEATURES)

encoder.save(ENCODER_PATH)

# Prepare to model training
### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model. It is required by the NN models to correctly create embeddings for every source column. Note that user_id does not required in `TensorSchema`.

Note that **cardinality** is the number of unique values ​in the item catalog (vocabulary). **Padding value** is the next value after the last one.

In [12]:
from replay.data import FeatureHint, FeatureType, FeatureSource
from replay.data.nn import TensorFeatureInfo, TensorFeatureSource, TensorSchema


EMBEDDING_DIM = 64

NUM_UNIQUE_ITEMS = len(encoder.mapping["item_id"])

tensor_schema = TensorSchema(
    [
        TensorFeatureInfo(
            name="item_id",
            is_seq=True,
            padding_value=NUM_UNIQUE_ITEMS,
            cardinality=NUM_UNIQUE_ITEMS,
            embedding_dim=EMBEDDING_DIM,
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.ITEM_ID,
            feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, "item_id")]
        )
    ]
)

### Configure ParquetModule and transformation pipelines

The `ParquetModule` class enables training of models on large datasets by reading data in batch-wise way. This class initialized with **paths to every data split, a metadata dict containing information about shape and padding value of every column and a dict of transforms**. `ParquetModule`'s  "transform pipelines" are stage-specific modules implementing additional preprocessing to be performed on batch level right before the forward pass.  

For SasRec model (User tower), RePlay provides a function that generates a sequence of appropriate transforms for each data split named **make_default_sasrec_transforms**.

Internally this function creates the following transforms:
1) Training:
    1. Create a target, which contains the shifted item sequence that represents the next item in the sequence (for the next item prediction task).
    2. Rename features to match it with expected format by the model during training.
    3. Unsqueeze target (*positive_labels*) and it's padding mask (*target_padding_mask*) for getting required shape of this tensors for loss computation.
    4. Group input features to be embed in expected format.

2) Validation/Inference:
    1. Rename/group features to match it with expected format by the model during valdiation/inference.

If a different set of transforms is required, you can create them yourself and submit them to the ParquetModule in the form of a dictionary where the key is the name of the split, and the value is the list of transforms. Available transforms are in the replay/nn/transforms/.

**Note:** One of the transforms for the training data prepares the initial sequence for the task of Next Item Prediction so it shifts the sequence of items. For the final sequence length to be correct, you need to set shape of item_id in metadata as **model sequence length + shift**. Default shift value is 1.

In [13]:
from replay.nn.transform.template import make_default_twotower_transforms

transforms = make_default_twotower_transforms(tensor_schema)

In [14]:
MAX_SEQ_LEN = 50

train_metadata = {
    "train": {
        "item_id": {"shape": MAX_SEQ_LEN + 1, "padding": tensor_schema["item_id"].padding_value},
    },
    "validate": {
        "item_id": {"shape": MAX_SEQ_LEN, "padding": tensor_schema["item_id"].padding_value},
        "ground_truth": {"shape": 1, "padding": -1}
    },
}

In [15]:
from replay.data.nn import ParquetModule

BATCH_SIZE = 32

parquet_module = ParquetModule(
    train_path=TRAIN_PATH,
    validate_path=VAL_PATH,
    batch_size=BATCH_SIZE,
    metadata=train_metadata,
    transforms=transforms,
)

/tmp/ipykernel_18423/1761019238.py:5: UserWarning: The following dataset paths aren't provided: test,predict. Make sure to disable these stages in your Lightning Trainer configuration.
  parquet_module = ParquetModule(


## Train model
### Create TwoTower model instance and run the training stage using lightning
We may now train the model using the Lightning trainer class. 

RePlay's implementation of TwoTower is designed in a modular, **block-based approach**. Instead of passing configuration parameters to the constructor, TwoTower is built by providing fully initialized components that makes the model more flexible and easier to extend. 


#### Default Configuration

Default TwoTower model may be created quickly via method *from_params*. Default model instance has CE loss, user tower is SasRec with original SasRec transformer layes and sum aggregated embeddings, item tower is a SwiGlU MLP block. Both towers use the same features.

In [16]:
from replay.nn.sequential import TwoTower
from replay.nn.sequential.twotower import FeaturesReader

NUM_BLOCKS = 2
NUM_HEADS = 2
DROPOUT = 0.3

twotower = TwoTower.from_params(
    schema=tensor_schema,
    embedding_dim=EMBEDDING_DIM,
    max_sequence_length=MAX_SEQ_LEN,
    num_heads=NUM_HEADS,
    num_blocks=NUM_BLOCKS,
    dropout=DROPOUT,
    item_features_reader=FeaturesReader(
        schema=tensor_schema,
        metadata={"item_id": {}},
        path=PATH_ENCODED_FEATURES,
    )
)

A universal PyTorch Lightning module is provided that can work with any NN model.

In [17]:
from replay.nn.lightning import LightningModule

model = LightningModule(twotower)

To facilitate training, we add the following callbacks:
1) `ModelCheckpoint` - to save the best trained model based on its Recall metric. It's a default Lightning Callback.
1) `ComputeMetricsCallback` - to display a detailed validation metric matrix after each epoch. It's a custom RePlay callback for computing recsys metrics on validation and test stages.


In [19]:
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from replay.nn.lightning.callback import ComputeMetricsCallback


checkpoint_callback = ModelCheckpoint(
    dirpath="twotower/checkpoints/",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ComputeMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=NUM_UNIQUE_ITEMS,
)

csv_logger = CSVLogger(save_dir="twotower/.logs/train", name="TwoTower-example")

trainer = L.Trainer(
    max_epochs=5,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

trainer.fit(model, datamodule=parquet_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | TwoTower | 352 K  | train
-------------------------------------------
352 K     Trainable params
0         Non-trainable params
352 K     Total params
1.409     Total estimated model params size (MB)
53        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 189: 'recall@10' reached 0.03460 (best 0.03460), saving model to '/home/RePlay/examples/twotower/checkpoints/epoch=0-step=189.ckpt' as top 1


...

Epoch 3, global step 756: 'recall@10' reached 0.14950 (best 0.14950), saving model to '/home/RePlay/examples/twotower/checkpoints/epoch=3-step=756.ckpt' as top 1


k            1         5         10        20
map     0.01606  0.040792  0.048443  0.055003
ndcg    0.01606  0.053037  0.071881  0.095889
recall  0.01606  0.090563  0.149503  0.244702 



Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 4, global step 945: 'recall@10' reached 0.16887 (best 0.16887), saving model to '/home/RePlay/examples/twotower/checkpoints/epoch=4-step=945.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


k             1         5         10        20
map     0.019205  0.045919  0.055054  0.061132
ndcg    0.019205  0.059209  0.081423  0.103788
recall  0.019205  0.100000  0.168874  0.257781 



Now we can get the best model path stored in the checkpoint callback.

In [20]:
best_model_path = checkpoint_callback.best_model_path
best_model_path

'/home/RePlay/examples/twotower/checkpoints/epoch=4-step=945.ckpt'

## Inference

To obtain model scores, we will load the weights from the best checkpoint. To do this, we use the `LightningModule`, provide there the path to the checkpoint and the model instance.

In [21]:
twotower = TwoTower.from_params(
    schema=tensor_schema,
    embedding_dim=EMBEDDING_DIM,
    max_sequence_length=MAX_SEQ_LEN,
    num_heads=NUM_HEADS,
    num_blocks=NUM_BLOCKS,
    dropout=DROPOUT,
    item_features_reader=FeaturesReader(
        schema=tensor_schema,
        metadata={"item_id": {}},
        path=PATH_ENCODED_FEATURES,
    )
)

best_model = LightningModule.load_from_checkpoint(best_model_path, model=twotower)
best_model.eval();

Configure `ParquetModule` for inference. It's necessary to read "user_id" column from data for correctly mapping predictions, so add "user_id" to metadata for `ParquetModule`. For reading non-array column dict should be empty.

In [22]:
inference_metadata = {
    "predict": {
        "user_id": {},
        "item_id": {"shape": MAX_SEQ_LEN, "padding": tensor_schema["item_id"].padding_value},
    }
}

parquet_module = ParquetModule(
    predict_path=TEST_PATH,
    batch_size=BATCH_SIZE,
    metadata=inference_metadata,
    transforms=transforms,
)

During inference, we can use `TopItemsCallback`. Such callback allows you to get scores for each user throughout the entire catalog and get recommendations in the form of ids of items with the highest score values.


Recommendations can be fetched in four formats: PySpark DataFrame, Pandas DataFrame, Polars DataFrame or raw PyTorch tensors. Each of the types corresponds a callback. In this example, we'll be using the `PandasTopItemsCallback`.

In [23]:
from replay.nn.lightning.callback import PandasTopItemsCallback

csv_logger = CSVLogger(save_dir="twotower/.logs/test", name="TwoTower-example")

TOPK = [1, 5, 10, 20]

pandas_prediction_callback = PandasTopItemsCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
)

trainer = L.Trainer(callbacks=[pandas_prediction_callback], logger=csv_logger, inference_mode=True)
trainer.predict(best_model, datamodule=parquet_module, return_predictions=False)

pandas_res = pandas_prediction_callback.get_result()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [24]:
pandas_res

,user_id,item_id,score
0,0,360,28.124413
0,0,773,27.929251
0,0,1526,27.875792
0,0,584,27.831373
0,0,1838,27.78071
...,...,...,...
6037,6039,504,29.747719
6037,6039,1079,29.701241
6037,6039,297,29.686306
6037,6039,333,29.681482


### Calculating metrics

*test_gt* is already encoded, so we can use it for computing metrics.

In [25]:
from replay.metrics import MAP, OfflineMetrics, Precision, Recall
from replay.metrics.torch_metrics_builder import metrics_to_df

In [26]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK)],
    query_column="user_id",
    rating_column="score",
)(pandas_res, test_gt.explode("item_id"))

In [27]:
metrics_to_df(result_metrics)

k,1,5,10,20
MAP,0.016893,0.042296,0.050793,0.057049
Precision,0.016893,0.018417,0.015750,0.012405
Recall,0.016893,0.092083,0.157502,0.248095


Let's call the `inverse_transform` encoder's function to get the final dataframe with recommendations

In [28]:
encoder.inverse_transform(pandas_res)

,user_id,item_id,score
0,1,364,28.124413
0,1,783,27.929251
0,1,1566,27.875792
0,1,588,27.831373
0,1,1907,27.78071
...,...,...,...
6037,6040,508,29.747719
6037,6040,1095,29.701241
6037,6040,300,29.686306
6037,6040,337,29.681482
